# Cancelling transactions in LUSID

This notebooks shows how you can use the [<b>cancel transactions</b>](https://www.lusid.com/docs/api/#operation/CancelTransactions) endpoint to cancel transactions in a LUSID portfolio. For the pruposes of this demo, we will first seed a portfolio with 21 transactions spanning cash and various FTSE 100 stocks. We will then cancel 1 <i> Aviva</i> transaction and 2 <i> BHP </i> transactions.

### Setup LUSID

In [1]:
# Import general purpose packages
import os

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

### 1) Load default transactions into a new scope

In [2]:
# Create a new scope

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [3]:
# Load a file of equity transactions

transactions_file = r"data/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [4]:
# Load portfolios, instruments, and transactions

seed_data = seed_data(api_factory,
          ["portfolios", "instruments", "transactions"],
          scope,
          transactions_file,
          "csv"
         )

In [5]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

### 2) Lets check our holdings

In [6]:
response = txn_port_api.get_holdings(scope=scope,
                                     code=portfolio_code,
                                     property_keys=["Instrument/default/Name"])
lusid_response_to_data_frame(response, rename_properties=True)

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,3855-5369-608c-7e/EQUITY_UK,P,137088.00,137088.00,549997.05,GBP,0.0,GBP
1,LUID_7XM08GZF,{},BHP,3855-5369-608c-7e/EQUITY_UK,P,115554.00,115554.00,2000008.64,GBP,0.0,GBP
2,LUID_STGB38I6,{},Barclays,3855-5369-608c-7e/EQUITY_UK,P,286238.00,286238.00,500000.54,GBP,0.0,GBP
3,LUID_PVOJGULG,{},BP,3855-5369-608c-7e/EQUITY_UK,P,203170.00,203170.00,1000002.74,GBP,0.0,GBP
4,LUID_Z57YKL4W,{},HSBC,3855-5369-608c-7e/EQUITY_UK,P,42698.00,42698.00,249996.80,GBP,0.0,GBP
5,LUID_RNHEK2PL,{},Morrisons,3855-5369-608c-7e/EQUITY_UK,P,372948.00,372948.00,749998.42,GBP,0.0,GBP
6,LUID_99M6G8U7,{},Tesco,3855-5369-608c-7e/EQUITY_UK,P,213676.00,213676.00,500001.84,GBP,0.0,GBP
7,LUID_6JES517Q,{},Rightmove,3855-5369-608c-7e/EQUITY_UK,P,159084.00,159084.00,1000002.02,GBP,0.0,GBP
8,LUID_7KLNIUU7,{},vodafone,3855-5369-608c-7e/EQUITY_UK,P,935980.00,935980.00,1500001.54,GBP,0.0,GBP
9,LUID_NE84MHW9,{},Anglo American plc,3855-5369-608c-7e/EQUITY_UK,P,72904.00,72904.00,1499999.80,GBP,0.0,GBP


### 3) What transactions make up our Aviva and BHP holdings?

In [7]:
response = txn_port_api.build_transactions(scope=scope,
                                           code=portfolio_code,
                                           query_parameters = models.TransactionQueryParameters(start_date="2020-01-01",
                                 end_date="2020-12-31") ,
                                           property_keys = ["Instrument/default/Name"])

In [8]:
lusid_response_to_data_frame(response, rename_properties=True).head(5)

,transaction_id,type,description,instrument_identifiers.Instrument/default/ClientInternal,instrument_uid,transaction_date,settlement_date,units,transaction_amount,transaction_price.price,...,transaction_currency,strategy(3855-5369-608c-7e-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding(default-Properties),Name(default-Properties),transaction_status,entry_date_time,realised_gain_loss,instrument_identifiers.Instrument/default/Currency
0,trd_0001,Buy,Purchase,EQ_1234,LUID_ATFGUBHS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,124626.0,499999.51,4.0120,...,GBP,ftse_tracker,27ffc53b-05f9-4251-83de-0e99a8f5a953,3855-5369-608c-7e,124626.0,Aviva,Active,2020-03-31 07:59:21.480680+00:00,[],NaN
1,trd_0002,Buy,Purchase,EQ_1234,LUID_ATFGUBHS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12462.0,49997.54,4.0120,...,GBP,ftse_tracker,27ffc53b-05f9-4251-83de-0e99a8f5a953,3855-5369-608c-7e,137088.0,Aviva,Active,2020-03-31 07:59:21.480680+00:00,[],NaN
2,trd_0003,Buy,Purchase,EQ_1235,LUID_7XM08GZF,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,57777.0,1000004.32,17.3080,...,GBP,ftse_tracker,27ffc53b-05f9-4251-83de-0e99a8f5a953,3855-5369-608c-7e,57777.0,BHP,Active,2020-03-31 07:59:21.480680+00:00,[],NaN
3,trd_0004,Buy,Purchase,EQ_1235,LUID_7XM08GZF,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,57777.0,1000004.32,17.3080,...,GBP,ftse_tracker,27ffc53b-05f9-4251-83de-0e99a8f5a953,3855-5369-608c-7e,115554.0,BHP,Active,2020-03-31 07:59:21.480680+00:00,[],NaN
4,trd_0005,Buy,Purchase,EQ_1236,LUID_STGB38I6,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,143119.0,250000.27,1.7468,...,GBP,ftse_tracker,27ffc53b-05f9-4251-83de-0e99a8f5a953,3855-5369-608c-7e,143119.0,Barclays,Active,2020-03-31 07:59:21.480680+00:00,[],NaN


### 4) Cancel one of the Aviva transactions

In [9]:
txn_port_api.cancel_transactions(scope=scope,
                                code = portfolio_code,
                                transaction_ids=["trd_0001"])

{'as_at': datetime.datetime(2020, 3, 31, 7, 59, 24, 811673, tzinfo=tzutc()),
 'effective_from': None,
 'href': None,
 'links': [{'description': 'Request ID',
            'href': 'http://tradermadetest.lusid.com/app/insights/logs/0HLUL65PDL0DC:00000003',
            'method': 'DELETE',
            'relation': 'RequestLogs'}]}

### 5) Cancel two of the BHP transactions

In [10]:
txn_port_api.cancel_transactions(scope=scope,
                                code = portfolio_code,
                                transaction_ids=["trd_0003", "trd_0004"])

{'as_at': datetime.datetime(2020, 3, 31, 7, 59, 25, 766355, tzinfo=tzutc()),
 'effective_from': None,
 'href': None,
 'links': [{'description': 'Request ID',
            'href': 'http://tradermadetest.lusid.com/app/insights/logs/0HLUL8N159VDV:00000004',
            'method': 'DELETE',
            'relation': 'RequestLogs'}]}

### 6) Check holdings again

We can see two updates:

* Holdings in Aviva have been reduced 
* We no longer have any BHP holdings

In [11]:
response = txn_port_api.get_holdings(scope=scope,
                                     code=portfolio_code,
                                     property_keys=["Instrument/default/Name"])
lusid_response_to_data_frame(response, rename_properties=True)

,instrument_uid,sub_holding_keys,Name(default-Properties),SourcePortfolioId(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency
0,LUID_ATFGUBHS,{},Aviva,3855-5369-608c-7e/EQUITY_UK,P,12462.00,12462.00,49997.54,GBP,0.0,GBP
1,LUID_STGB38I6,{},Barclays,3855-5369-608c-7e/EQUITY_UK,P,286238.00,286238.00,500000.54,GBP,0.0,GBP
2,LUID_PVOJGULG,{},BP,3855-5369-608c-7e/EQUITY_UK,P,203170.00,203170.00,1000002.74,GBP,0.0,GBP
3,LUID_Z57YKL4W,{},HSBC,3855-5369-608c-7e/EQUITY_UK,P,42698.00,42698.00,249996.80,GBP,0.0,GBP
4,LUID_RNHEK2PL,{},Morrisons,3855-5369-608c-7e/EQUITY_UK,P,372948.00,372948.00,749998.42,GBP,0.0,GBP
5,LUID_99M6G8U7,{},Tesco,3855-5369-608c-7e/EQUITY_UK,P,213676.00,213676.00,500001.84,GBP,0.0,GBP
6,LUID_6JES517Q,{},Rightmove,3855-5369-608c-7e/EQUITY_UK,P,159084.00,159084.00,1000002.02,GBP,0.0,GBP
7,LUID_7KLNIUU7,{},vodafone,3855-5369-608c-7e/EQUITY_UK,P,935980.00,935980.00,1500001.54,GBP,0.0,GBP
8,LUID_NE84MHW9,{},Anglo American plc,3855-5369-608c-7e/EQUITY_UK,P,72904.00,72904.00,1499999.80,GBP,0.0,GBP
9,CCY_GBP,{},CCY_GBP,3855-5369-608c-7e/EQUITY_UK,B,4949998.76,4949998.76,4949998.76,GBP,0.0,GBP
